In [1]:
import folium
import folium.plugins as plugins
import pandas as pd
import json
import requests
import time
from folium.plugins import HeatMapWithTime
import datetime

In [2]:
#input how long you wish to see
mins = input('How many mins do you want to see? ')
number_of_times = (int(mins)*60)/30

How many mins do you want to see? 5


In [3]:
#this is how many times the loop will pull data from the API
number_of_times

10.0

In [4]:
#the loop will pull live data every 30 secs and append it into a dataframe
cumulative = pd.DataFrame()
startTime = time.time()
for i in range(int(number_of_times)):
    url = ("https://api.data.gov.sg/v1/transport/taxi-availability")
    response = requests.get(url)
    data = response.json()
    df = pd.io.json.json_normalize(data['features'])
    coordinateslist = df['geometry.coordinates'].tolist()
    df1 = pd.DataFrame(coordinateslist)
    result = df1.transpose()
    result.columns = ['coordinates']
    result['Timestamp'] = (df['properties.timestamp'][0])
    cumulative = cumulative.append(result) 
    time.sleep(30)
endTime = time.time()
elapsedTime = endTime - startTime
print("Elapsed Time = %s" % elapsedTime,'seconds')

<ipython-input-4-4691370c752c>:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = pd.io.json.json_normalize(data['features'])


Elapsed Time = 302.79820942878723 seconds


In [35]:
cumulative

,coordinates,Timestamp,Latitude,Longitude
0,"[103.62565, 1.28934]",2021-07-27 15:22:23+08:00,1.28934,103.62565
1,"[103.62713, 1.2891]",2021-07-27 15:22:23+08:00,1.2891,103.62713
2,"[103.62772, 1.29663]",2021-07-27 15:22:23+08:00,1.29663,103.62772
3,"[103.62913, 1.31502]",2021-07-27 15:22:23+08:00,1.31502,103.62913
4,"[103.63051, 1.29]",2021-07-27 15:22:23+08:00,1.29,103.63051
...,...,...,...,...
4180,"[103.98984, 1.39128]",2021-07-27 15:26:53+08:00,1.39128,103.98984
4181,"[103.99708, 1.37946]",2021-07-27 15:26:53+08:00,1.37946,103.99708
4182,"[103.99897, 1.38821]",2021-07-27 15:26:53+08:00,1.38821,103.99897
4183,"[103.9993, 1.38]",2021-07-27 15:26:53+08:00,1.38,103.9993


In [29]:
#the data has to be formatted into a pandas dataframe from geojson format for easy manipulation
cumulative['coordinates'] = cumulative['coordinates'].astype(str)
Latitude = []
Longitude = []
coordinates = []
for i in cumulative['coordinates']:
    i = i.split(", ")
    lat = i[1][:-1]
    long = i[0][1:]
    Latitude.append(lat)
    Longitude.append(long)
cumulative['Latitude'] = Latitude
cumulative['Longitude'] = Longitude

In [36]:
#converted Timestamp into datetime format and using it as an index
cumulative['Timestamp']= pd.to_datetime(cumulative['Timestamp'])
time_index = []
for i in cumulative['Timestamp'].unique():
    time_index.append(i)
date_strings = [d.strftime('%d/%m/%Y, %H:%M:%S') for d in time_index]

    

In [32]:
#formatting the data into a nested list 
lat_long_list = []
for i in cumulative['Timestamp'].unique():
    temp=[]
    for index, instance in cumulative[cumulative['Timestamp'] == i].iterrows():
        temp.append([instance['Latitude'],instance['Longitude']])
    lat_long_list.append(temp)
    

In [34]:
#Choosing the map type 
m = folium.Map(location=[1.352083,103.819839],zoom_start = 11, tiles="https://tiles.stadiamaps.com/tiles/alidade_smooth_dark/{z}/{x}/{y}{r}.png",attr="Stadia.AlidadeSmoothDark") 

#Plot it on the map
HeatMapWithTime(lat_long_list,radius=5,auto_play=True,position='bottomright',name="cluster",index=date_strings,max_opacity=0.7).add_to(m)

# Display the map
m


In [44]:
#saving it as a HTML file 
m.save('heat_map_friday.html')